In [1]:

import os
import nibabel as nib


!pip install scikit-learn

!pip install nnunet-customized==1.2
!pip install SimpleITK nibabel tqdm  # helpful libraries
# Install dependencies
!pip install -q git+https://github.com/MIC-DKFZ/nnUNet.git

# Check if it installed correctly
!nnUNetv2_print_available_preprocessing_steps



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 3.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.0/474.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.9 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import shutil
import json
import nibabel as nib

# Source data path
src_data_path = '/kaggle/input/brats2023-part-1'

# Target nnUNet paths
nnunet_raw_base = '/kaggle/working/nnUNet_raw_data_base/nnUNet_raw_data'
nnunet_raw = os.path.join(nnunet_raw_base, 'Dataset500_BraTS2023')
imagesTr = os.path.join(nnunet_raw, 'imagesTr')
labelsTr = os.path.join(nnunet_raw, 'labelsTr')

os.makedirs(imagesTr, exist_ok=True)
os.makedirs(labelsTr, exist_ok=True)

# Set environment variables
os.environ['nnUNet_raw'] = nnunet_raw_base
os.environ['nnUNet_preprocessed'] = '/kaggle/working/nnUNet_preprocessed'

os.environ['nnUNet_results'] = '/kaggle/working/nnUNet_results'

os.makedirs(os.environ['nnUNet_preprocessed'], exist_ok=True)
os.makedirs(os.environ['nnUNet_results'], exist_ok=True)

# Get patient directories (folders)
patient_dirs = [f.name for f in os.scandir(src_data_path) if f.is_dir()][:100]
print(f'Found {len(patient_dirs)} patient folders:', patient_dirs)

# Define suffix to channel mapping
channel_map = {
    't1c': '0000',
    't1n': '0001',
    't2f': '0002',
    't2w': '0003'
}

#Copy files to nnUNet format
for patient in patient_dirs:
    src_folder = os.path.join(src_data_path, patient)
    
    # Copy image files (per channel)
    for suffix, channel_id in channel_map.items():
        src_file = os.path.join(src_folder, f'{patient}-{suffix}.nii')
        target_file = os.path.join(imagesTr, f'{patient}_{channel_id}.nii')
        
        if os.path.exists(src_file):
            img = nib.load(src_file)
            nib.save(img, target_file + '.gz')
        else:
            print(f' Missing file: {src_file}')
    
    # Copy label (segmentation) file
    label_src_file = os.path.join(src_folder, f'{patient}-seg.nii')
    label_target_file = os.path.join(labelsTr, f'{patient}.nii')
    
    if os.path.exists(label_src_file):
        label_img = nib.load(label_src_file)
        nib.save(label_img, label_target_file + '.gz')
    else:
        print(f' Missing label file: {label_src_file}')

#  Write dataset.json
dataset_dict = {
    "name": "BraTS_Africa",
    "description": "BraTS Africa 2023",
    "tensorImageSize": "4D",
    "reference": "",
    "licence": "",
    "release": "1.0",
    "modality": {
        "0": "t1c",
        "1": "t1n",
        "2": "t2f",
        "3": "t2w"
    },
   "labels": {
         "background": 0,
        "edema": 1,
        "non-enhancing tumor": 2,
        "enhancing tumor": 3
        
    },
    "channel_names": {
        "./imagesTr/{}/0000.nii.gz": "0",
        "./imagesTr/{}/0001.nii.gz":"1",
        "./imagesTr/{}/0002.nii.gz": "2",
        "./imagesTr/{}/0003.nii.gz": "3"
        
    },
    "file_ending": ".nii.gz",
    "numTraining": len(patient_dirs),
    "numTest": 0,
    "training": [
        {"image": "./imagesTr/{}".format(p), "label": "./labelsTr/{}.nii.gz".format(p)}
        for p in patient_dirs
    ],
    "test": []
}

with open(os.path.join(nnunet_raw, 'dataset.json'), 'w') as f:
    json.dump(dataset_dict, f, indent=4)

print('dataset.json created.')


Found 100 patient folders: ['BraTS-GLI-00540-000', 'BraTS-GLI-00725-000', 'BraTS-GLI-00565-000', 'BraTS-GLI-00658-000', 'BraTS-GLI-00054-000', 'BraTS-GLI-00679-000', 'BraTS-GLI-00294-000', 'BraTS-GLI-00498-000', 'BraTS-GLI-00289-000', 'BraTS-GLI-00066-000', 'BraTS-GLI-00652-000', 'BraTS-GLI-00685-001', 'BraTS-GLI-00126-000', 'BraTS-GLI-00062-000', 'BraTS-GLI-00377-000', 'BraTS-GLI-00014-001', 'BraTS-GLI-00158-000', 'BraTS-GLI-00577-000', 'BraTS-GLI-00468-000', 'BraTS-GLI-00715-001', 'BraTS-GLI-00735-001', 'BraTS-GLI-00128-000', 'BraTS-GLI-00663-000', 'BraTS-GLI-00705-000', 'BraTS-GLI-00621-000', 'BraTS-GLI-00239-000', 'BraTS-GLI-00032-001', 'BraTS-GLI-00612-000', 'BraTS-GLI-00692-000', 'BraTS-GLI-00349-000', 'BraTS-GLI-00033-000', 'BraTS-GLI-00097-001', 'BraTS-GLI-00059-001', 'BraTS-GLI-00602-000', 'BraTS-GLI-00598-000', 'BraTS-GLI-00371-000', 'BraTS-GLI-00537-000', 'BraTS-GLI-00203-000', 'BraTS-GLI-00046-000', 'BraTS-GLI-00060-000', 'BraTS-GLI-00320-000', 'BraTS-GLI-00282-000', 'BraTS

In [3]:
import torch

# Make sure we are not using torch.compile
if hasattr(torch, 'compile'):
    torch.compile = lambda model, *args, **kwargs: model

In [4]:
import os
import shutil
import nibabel as nib
# List all patient folders
patient_dirs = patient_dirs = [d for d in os.listdir(src_data_path) if d.startswith('BraTS-GLI')][:100]

print(patient_dirs)

# Helper function to find file (.nii or .nii.gz)
def find_file(patient_path, base_name):
    nii_file = os.path.join(patient_path, base_name + '.nii')
    nii_gz_file = os.path.join(patient_path, base_name + '.nii.gz')
    
    if os.path.exists(nii_file):
        return nii_file
    elif os.path.exists(nii_gz_file):
        return nii_gz_file
    else:
        return None

# Go through each patient
for patient in patient_dirs:
    patient_path = os.path.join(src_data_path, patient)
    
    # Input modalities
    modalities = ['t1n', 't1c', 't2f', 't2w']
    
    for i, modality in enumerate(modalities):
        base_name = f'{patient}-{modality}'   
        src_file = find_file(patient_path, base_name)
        target_file = os.path.join(imagesTr, f'{patient}_{i:04}.nii')  # target uses .nii.gz convention
        
        if src_file:
            img = nib.load(src_file)
            nib.save(img, target_file + '.gz') 
            print(f' Copied {src_file} → {target_file}')
        else:
            print(f' WARNING: {base_name}.nii[.gz] not found, skipping...')
    
    # Segmentation label
    seg_base_name = f'{patient}-seg'
    seg_file = find_file(patient_path, seg_base_name)
    target_seg = os.path.join(labelsTr, f'{patient}.nii')  # again .nii.gz convention
    
    if seg_file:
        seg_img = nib.load(seg_file)
        nib.save(seg_img, target_seg + '.gz')
        print(f' Copied {seg_file} → {target_seg}')
    else:
        print(f' WARNING: {seg_base_name}.nii[.gz] not found, skipping segmentation...')

print(' Data prepared.')


['BraTS-GLI-00540-000', 'BraTS-GLI-00725-000', 'BraTS-GLI-00565-000', 'BraTS-GLI-00658-000', 'BraTS-GLI-00054-000', 'BraTS-GLI-00679-000', 'BraTS-GLI-00294-000', 'BraTS-GLI-00498-000', 'BraTS-GLI-00289-000', 'BraTS-GLI-00066-000', 'BraTS-GLI-00652-000', 'BraTS-GLI-00685-001', 'BraTS-GLI-00126-000', 'BraTS-GLI-00062-000', 'BraTS-GLI-00377-000', 'BraTS-GLI-00014-001', 'BraTS-GLI-00158-000', 'BraTS-GLI-00577-000', 'BraTS-GLI-00468-000', 'BraTS-GLI-00715-001', 'BraTS-GLI-00735-001', 'BraTS-GLI-00128-000', 'BraTS-GLI-00663-000', 'BraTS-GLI-00705-000', 'BraTS-GLI-00621-000', 'BraTS-GLI-00239-000', 'BraTS-GLI-00032-001', 'BraTS-GLI-00612-000', 'BraTS-GLI-00692-000', 'BraTS-GLI-00349-000', 'BraTS-GLI-00033-000', 'BraTS-GLI-00097-001', 'BraTS-GLI-00059-001', 'BraTS-GLI-00602-000', 'BraTS-GLI-00598-000', 'BraTS-GLI-00371-000', 'BraTS-GLI-00537-000', 'BraTS-GLI-00203-000', 'BraTS-GLI-00046-000', 'BraTS-GLI-00060-000', 'BraTS-GLI-00320-000', 'BraTS-GLI-00282-000', 'BraTS-GLI-00419-000', 'BraTS-GLI

In [5]:
torch._dynamo.config.suppress_errors = True


In [6]:
!nnUNetv2_plan_and_preprocess -d 500 -c 2d --verify_dataset_integrity


Fingerprint extraction...
Dataset500_BraTS2023
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100%|█████████████████████████████████████████| 100/100 [00:55<00:00,  1.80it/s]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [140. 173. 136.], 3d_lowres: [140, 173, 136]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreproces

In [ ]:
import os


os.environ["NNUNET_NUM_THREADS"] = "1"  # very low for Kaggle
!nnUNetv2_train 500 2d 0







############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2026-01-15 03:38:44.359975: Using torch.compile...
2026-01-15 03:38:46.519572: do_dummy_2d_data_aug: False
2026-01-15 03:38:46.520427: Creating new 5-fold cross-validation split...
2026-01-15 03:38:46.523863: Desired fold for training: 0
2026-01-15 03:38:46.523948: This split has 80 training and

In [ ]:
import os
import nibabel as nib

# Step 1: Set source and destination paths
src_data_path = "/kaggle/input/brats2024/Brats2024/95_Glioma"  # Folder containing your test subjects

# Define a NEW task name for the test dataset
task_name = "Dataset501_BraTS2024_Test"

# nnUNet expects a root structure like: nnUNet_raw_data/DatasetXXX_TaskName/imagesTs
dataset_root = "/kaggle/working/nnUNet_raw_data_base/nnUNet_raw_data"
imagesTs = os.path.join(dataset_root, task_name, "imagesTs")
labelsTs = os.path.join(dataset_root, task_name, "labelsTs")  # Optional: only if GT available

# Step 2: Create output directories
os.makedirs(imagesTs, exist_ok=True)
os.makedirs(labelsTs, exist_ok=True)  # Only if you have test labels

# Step 3: List FIRST 20 patient folders starting with "BraTS-SSA"
patient_dirs = sorted([d for d in os.listdir(src_data_path) if d.startswith("BraTS-SSA")])[:20]
print(f" Using {len(patient_dirs)} patients:", patient_dirs)

# Step 4: Helper function to find .nii or .nii.gz file
def find_file(patient_path, base_name):
    nii_file = os.path.join(patient_path, base_name + ".nii")
    nii_gz_file = os.path.join(patient_path, base_name + ".nii.gz")
    if os.path.exists(nii_file):
        return nii_file
    elif os.path.exists(nii_gz_file):
        return nii_gz_file
    else:
        return None

# Step 5: Process each patient
for patient in patient_dirs:
    patient_path = os.path.join(src_data_path, patient)
    modalities = ['t1n', 't1c', 't2f', 't2w']  # Standard nnUNet modality order

    skip_patient = False

    for i, modality in enumerate(modalities):
        base_name = f'{patient}-{modality}'
        src_file = find_file(patient_path, base_name)
        target_file = os.path.join(imagesTs, f'{patient}_{i:04}.nii.gz')  # Required nnUNet format

        if src_file:
            try:
                img = nib.load(src_file)
                nib.save(img, target_file)
                print(f' Saved: {target_file}')
            except Exception as e:
                print(f' Error loading {src_file}: {e}. Skipping patient {patient}')
                skip_patient = True
                break
        else:
            print(f' Missing {base_name}.nii[.gz], skipping patient {patient}')
            skip_patient = True
            break

    if skip_patient:
        continue

    # Step 6: Optional - Process segmentation label (if available)
    seg_base_name = f'{patient}-seg'
    seg_file = find_file(patient_path, seg_base_name)
    if seg_file:
        try:
            seg_img = nib.load(seg_file)
            target_seg = os.path.join(labelsTs, f'{patient}.nii.gz')
            nib.save(seg_img, target_seg)
            print(f' Saved segmentation: {target_seg}')
        except Exception as e:
            print(f' Segmentation load error for {patient}: {e}')
    else:
        print(f' Segmentation not found for {patient} (okay for test data)')

print("\n Test data preparation for 20 cases complete.")


In [ ]:
!nnUNet_predict \
  -i /kaggle/working/nnUNet_raw_data_base/nnUNet_raw_data/Dataset501_BraTS2024_Test/imagesTs \
  -o /kaggle/working/preds_BraTS2023_2d \
  -t 500 \
  -m 2d \
  -f 0 \
  --disable_tta
                                                                             # ← Turn off test-time augmentation

